# Arvak Core: Quantum Circuit Compilation Platform

This notebook demonstrates Arvak's core Python API without requiring any external quantum frameworks.

Arvak is a Rust-native quantum compilation platform that provides:
- Fast circuit construction with fluent API
- OpenQASM 3.0 import/export
- Advanced compilation and optimization
- Pre-built circuits (Bell, GHZ, QFT)
- Support for multiple hardware backends

## Setup and Imports

First, let's import Arvak and check the available integrations.

In [ ]:
import arvak

# Check available framework integrations
print("Available integrations:")
status = arvak.integration_status()
for name, info in status.items():
    icon = "✓" if info['available'] else "✗"
    print(f"  {icon} {name}: {info['packages']}")

if not status:
    print("  (No integrations installed - this is fine for core Arvak usage!)")

## Creating Circuits with Arvak's Fluent API

Arvak provides a fluent, chainable API for building quantum circuits.

In [ ]:
# Create a Bell state circuit
bell = arvak.Circuit("bell", num_qubits=2, num_clbits=2)
bell.h(0).cx(0, 1).measure_all()

print(f"Circuit: {bell.name}")
print(f"Qubits: {bell.num_qubits}, Classical bits: {bell.num_clbits}")
print(f"Circuit depth: {bell.depth()}")
print(f"Gate count: {bell.size()}")

## OpenQASM 3.0 Export

Arvak uses OpenQASM 3.0 as its interchange format, making it compatible with any tool that supports QASM.

In [ ]:
# Export to OpenQASM 3.0
qasm_str = arvak.to_qasm(bell)
print("OpenQASM 3.0 representation:")
print(qasm_str)

## Pre-Built Circuits

Arvak provides convenient constructors for common quantum circuits.

In [ ]:
# Bell state
bell_circuit = arvak.Circuit.bell()
print("Bell state circuit:")
print(arvak.to_qasm(bell_circuit))
print()

In [ ]:
# GHZ state (5 qubits)
ghz_5 = arvak.Circuit.ghz(5)
print(f"GHZ-5 state: {ghz_5.num_qubits} qubits, depth {ghz_5.depth()}")
print(arvak.to_qasm(ghz_5))
print()

In [ ]:
# Quantum Fourier Transform (4 qubits)
qft_4 = arvak.Circuit.qft(4)
print(f"QFT-4 circuit: {qft_4.num_qubits} qubits, depth {qft_4.depth()}")
print(arvak.to_qasm(qft_4))

## Importing from OpenQASM 3.0

You can import circuits from OpenQASM 3.0 strings or files.

In [ ]:
# Define a QASM circuit
qasm_input = '''
OPENQASM 3.0;
include "stdgates.inc";

qubit[3] q;
bit[3] c;

// Create GHZ-3 state
h q[0];
cx q[0], q[1];
cx q[1], q[2];
c = measure q;
'''

# Import into Arvak
imported_circuit = arvak.from_qasm(qasm_input)
print(f"Imported circuit:")
print(f"  Qubits: {imported_circuit.num_qubits}")
print(f"  Classical bits: {imported_circuit.num_clbits}")
print(f"  Depth: {imported_circuit.depth()}")

## Compilation Configuration

Arvak provides powerful compilation tools for targeting specific hardware architectures.

In [ ]:
# Define hardware constraints
from arvak import CouplingMap, BasisGates, PropertySet, Layout

# IQM backend configuration
coupling_map = CouplingMap.star(5)      # Star topology with 5 qubits
basis_gates = BasisGates.iqm()          # IQM native gates (PRX, CZ)
layout = Layout.trivial(5)              # 1:1 logical-to-physical mapping

# Build property set
props = PropertySet().with_target(coupling_map, basis_gates)
props.set_layout(layout)

print("IQM Backend Configuration:")
print(f"  Qubits: {coupling_map.num_qubits}")
print(f"  Topology: star")
print(f"  Edges: {coupling_map.edges()}")
print(f"  Basis gates: {basis_gates.gates()}")

## Comparing Different Backend Configurations

In [ ]:
# Compare different backend configurations
backends_info = [
    ("IQM", BasisGates.iqm(), CouplingMap.star(5)),
    ("IBM", BasisGates.ibm(), CouplingMap.linear(5)),
    ("Simulator", BasisGates.universal(), CouplingMap.full(5))
]

print("Backend Comparison:")
print("=" * 60)
for name, gates, topology in backends_info:
    print(f"\n{name} Backend:")
    print(f"  Native gates: {gates.gates()}")
    print(f"  Topology: {topology.edges()}")
    print(f"  Qubits: {topology.num_qubits}")

## Exporting for Execution

Arvak circuits can be exported and executed using the Arvak CLI.

In [ ]:
# Export circuit for CLI execution
output_qasm = arvak.to_qasm(bell)

# Save to file
with open("bell_circuit.qasm", "w") as f:
    f.write(output_qasm)

print("Circuit exported to bell_circuit.qasm")
print("\nTo execute this circuit:")
print("\n1. Simulator:")
print("   $ arvak run bell_circuit.qasm --backend sim --shots 1000")
print("\n2. IQM Backend (requires IQM_TOKEN):")
print("   $ export IQM_TOKEN=your_token_here")
print("   $ arvak run bell_circuit.qasm --backend iqm --shots 1000")
print("\n3. IBM Quantum (requires IBM_QUANTUM_TOKEN):")
print("   $ export IBM_QUANTUM_TOKEN=your_token_here")
print("   $ arvak run bell_circuit.qasm --backend ibm --shots 1000")

## Complex Circuit Example

Let's create a more complex circuit and analyze its properties.

In [ ]:
# Create a more complex circuit
complex_circuit = arvak.Circuit("complex", num_qubits=4, num_clbits=4)

# Build a circuit with various gates
complex_circuit.h(0)
complex_circuit.cx(0, 1)
complex_circuit.cx(1, 2)
complex_circuit.cx(2, 3)
complex_circuit.h(3)
complex_circuit.cx(3, 2)
complex_circuit.measure_all()

print("Complex Circuit Properties:")
print(f"  Qubits: {complex_circuit.num_qubits}")
print(f"  Depth: {complex_circuit.depth()}")
print(f"  Gate count: {complex_circuit.size()}")
print("\nQASM representation:")
print(arvak.to_qasm(complex_circuit))

## Next Steps

### Framework Integrations

If you want to use Arvak with other quantum frameworks:

- **Qiskit**: See `02_qiskit_integration.ipynb`
  - Install: `pip install arvak[qiskit]`
  - Features: Qiskit ↔ Arvak conversion, Arvak as Qiskit backend

- **Qrisp**: See `03_qrisp_integration.ipynb`
  - Install: `pip install arvak[qrisp]`
  - Features: High-level quantum programming with Arvak compilation

- **Cirq**: See `04_cirq_integration.ipynb`
  - Install: `pip install arvak[cirq]`
  - Features: Google Cirq with Arvak backends

### CLI Tools

Arvak provides a powerful command-line interface:

```bash
# Run circuits on simulators or real hardware
arvak run circuit.qasm --backend sim --shots 1000

# Compile and optimize circuits
arvak compile circuit.qasm --target iqm --optimize 3

# Visualize circuits
arvak viz circuit.qasm

# Launch interactive dashboard
arvak dashboard
```

### Documentation

- GitHub: https://github.com/hiq-lab/HIQ
- Issues: https://github.com/hiq-lab/HIQ/issues